In [1]:
import re

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import unicodedata

### Tareas.xlsx -> Tareas-limpio.xlsx(Proyectos)

In [2]:
print("\n=== INICIO DE LIMPIEZA DE TAREAS ===")

# 1. Carga inicial
print("\n1. CARGANDO ARCHIVO DE TAREAS")
print("----------------------------")
try:
    df_tareas = pd.read_excel('../Sucia/Tareas.xlsm', engine="openpyxl")
    print("✓ Archivo cargado exitosamente")
    print(f"- Filas iniciales: {df_tareas.shape[0]}")
    print(f"- Columnas iniciales: {df_tareas.shape[1]}")
    print("\nColumnas disponibles:")
    for col in df_tareas.columns:
        print(f"- {col}")
except Exception as e:
    print(f"❌ Error al cargar archivo: {e}")
    raise




=== INICIO DE LIMPIEZA DE TAREAS ===

1. CARGANDO ARCHIVO DE TAREAS
----------------------------
✓ Archivo cargado exitosamente
- Filas iniciales: 8288
- Columnas iniciales: 15

Columnas disponibles:
- PROYECTO
- TAREA
- DESCRIPCION
- SECCION
- FECHA
- PERIODICIDAD
- ORDEN
- RESPONSABLE
- CODPROY
- CODTAREA
- ETIQUETA
- CODIGO
- TIPO
- ORDEN.1
- CODCLI2


In [3]:
# 2. Verificación inicial de nulos
print("\n2. VERIFICACIÓN INICIAL DE NULOS")
print("------------------------------")
nulos_iniciales = df_tareas[['CODIGO', 'PROYECTO']].isnull().sum()
print("Nulos por columna:")
for columna, cantidad in nulos_iniciales.items():
  print(f"- {columna}: {cantidad} nulos")


2. VERIFICACIÓN INICIAL DE NULOS
------------------------------
Nulos por columna:
- CODIGO: 7002 nulos
- PROYECTO: 6994 nulos


In [4]:
print("\n3. LIMPIEZA DE CODIGO")
print("-------------------")

# Analizar y clasificar los problemas en CODIGO
def analizar_codigo(codigo):
    if pd.isnull(codigo):
        return "NaN o valor nulo"
    if not isinstance(codigo, (int, float, str)):
        return f"Tipo inválido: {type(codigo)}"
    codigo_str = str(codigo).replace('.', '')  # Eliminar puntos para evaluar si es numérico
    if not codigo_str.isdigit():
        return f"Valor no numérico: {codigo}"
    return None

# Agregar columna con problemas identificados
df_tareas['PROBLEMA_CODIGO'] = df_tareas['CODIGO'].apply(analizar_codigo)

# Filtrar registros con problemas
df_tareas_invalidas = df_tareas[df_tareas['PROBLEMA_CODIGO'].notnull()]

# Mostrar resumen de problemas
print(f"\nRegistros con CODIGO inválido encontrados: {len(df_tareas_invalidas)}")

# Mostrar detalles específicos
if not df_tareas_invalidas.empty:
    print("\nTipos de problemas encontrados:")
    print(df_tareas_invalidas['PROBLEMA_CODIGO'].value_counts())

    print("\nEjemplos de registros con problemas:")
    print(df_tareas_invalidas[['CODIGO', 'PROYECTO', 'PROBLEMA_CODIGO']].head(20))

# Proceder con la limpieza una vez analizados los problemas
print("\nProcediendo con la limpieza...")

# Eliminar registros con problemas en CODIGO
filas_iniciales = len(df_tareas)
df_tareas['CODIGO'] = pd.to_numeric(df_tareas['CODIGO'], errors='coerce')
df_tareas = df_tareas.dropna(subset=['CODIGO'])
df_tareas['CODIGO'] = df_tareas['CODIGO'].astype(int)
filas_finales = len(df_tareas)

print(f"\nResultados de limpieza:")
print(f"- Filas iniciales: {filas_iniciales}")
print(f"- Filas eliminadas: {filas_iniciales - filas_finales}")
print(f"- Filas finales: {filas_finales}")

# Seleccionar columnas finales
df_tareas_filtrado = df_tareas[['CODIGO', 'PROYECTO']].copy()
print(f"Columnas seleccionadas: {list(df_tareas_filtrado.columns)}")



3. LIMPIEZA DE CODIGO
-------------------

Registros con CODIGO inválido encontrados: 7002

Tipos de problemas encontrados:
PROBLEMA_CODIGO
NaN o valor nulo    7002
Name: count, dtype: int64

Ejemplos de registros con problemas:
      CODIGO                      PROYECTO   PROBLEMA_CODIGO
762      NaN            Visitas Periodicas  NaN o valor nulo
763      NaN    🗞️ Visitas periodicas Hugo  NaN o valor nulo
839      NaN  🗞️ Visitas periodicas Martin  NaN o valor nulo
901      NaN   🗞️ Visitas Periodicas Dario  NaN o valor nulo
1125     NaN         Clientes Especiales 2  NaN o valor nulo
1126     NaN                      Creditos  NaN o valor nulo
1130     NaN                💲Creditos Hugo  NaN o valor nulo
1230     NaN               💲Creditos Dario  NaN o valor nulo
1294     NaN                           NaN  NaN o valor nulo
1295     NaN                           NaN  NaN o valor nulo
1296     NaN                           NaN  NaN o valor nulo
1297     NaN                          

In [5]:
# 4. Limpieza de PROYECTO
print("\n4. LIMPIEZA DE PROYECTO")
print("---------------------")
# Función para limpiar texto
def strip_accents(text):
  if pd.isna(text):
      return text
  return ''.join(c for c in unicodedata.normalize('NFD', str(text))
                if unicodedata.category(c) != 'Mn')

# Aplicar limpieza a PROYECTO
df_tareas['PROYECTO'] = df_tareas['PROYECTO'].apply(strip_accents)
print("✓ Acentos removidos de la columna PROYECTO")


4. LIMPIEZA DE PROYECTO
---------------------
✓ Acentos removidos de la columna PROYECTO


In [6]:
# 5. Selección final de columnas
print("\n5. SELECCIÓN DE COLUMNAS")
print("----------------------")
columnas_finales = ['CODIGO', 'PROYECTO']
df_tareas = df_tareas[columnas_finales]
print(f"Columnas seleccionadas: {', '.join(columnas_finales)}")



5. SELECCIÓN DE COLUMNAS
----------------------
Columnas seleccionadas: CODIGO, PROYECTO


In [7]:
# 6. Guardado de archivo
print("\n6. GUARDANDO ARCHIVO LIMPIO")
print("-------------------------")
try:
  df_tareas.to_excel("../Limpia/Tareas-limpio.xlsx", index=False)
  print("✓ Archivo guardado exitosamente en '../Limpia/Tareas-limpio.xlsx'")
  
  # Verificación final
  print("\nEstado final del DataFrame:")
  print(f"- Filas: {df_tareas.shape[0]}")
  print(f"- Columnas: {df_tareas.shape[1]}")
  print("\nMuestra de datos limpios:")
  print(df_tareas.sample(min(5, len(df_tareas))))
  
  print("\nVerificación final de nulos:")
  nulos_finales = df_tareas.isnull().sum()
  for columna, cantidad in nulos_finales.items():
      print(f"- {columna}: {cantidad} nulos")
except Exception as e:
  print(f"❌ Error al guardar archivo: {e}")

print("\n=== FIN DE LIMPIEZA DE TAREAS ===")


6. GUARDANDO ARCHIVO LIMPIO
-------------------------
✓ Archivo guardado exitosamente en '../Limpia/Tareas-limpio.xlsx'

Estado final del DataFrame:
- Filas: 1286
- Columnas: 2

Muestra de datos limpios:
      CODIGO                    PROYECTO
171    81233          Ruta Dario Viernes
1023   12500         Clientes Especiales
1192   41846            💲Creditos Martin
642    80261    Ruta Alejandro Miercoles
899    80266  🗞 Visitas Periodicas Dario

Verificación final de nulos:
- CODIGO: 0 nulos
- PROYECTO: 0 nulos

=== FIN DE LIMPIEZA DE TAREAS ===


In [8]:
df_tareas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1286 entries, 0 to 1293
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   CODIGO    1286 non-null   int64 
 1   PROYECTO  1286 non-null   object
dtypes: int64(1), object(1)
memory usage: 30.1+ KB


In [9]:
valores_unicos = df_tareas['PROYECTO'].unique()

# Imprimir los valores únicos
print(valores_unicos)

['Ruta Dario Lunes' 'Ruta Dario Martes' 'Ruta Dario Miercoles'
 'Ruta Dario Jueves' 'Ruta Dario Viernes' 'Ruta Dario Sabado'
 'Ruta Hugo Lunes' 'Ruta Hugo Martes' 'Ruta Hugo Miercoles'
 'Ruta Hugo Jueves' 'Ruta Hugo Viernes' 'Ruta Hugo Sabado'
 'Ruta Martin Lunes' 'Ruta Martin Martes' 'Ruta Martin Miercoles'
 'Ruta Martin Jueves' 'Ruta Martin Viernes' 'Ruta Martin Sabado'
 'Ruta Alejandro Lunes' 'Ruta Alejandro Martes' 'Ruta Alejandro Miercoles'
 'Ruta Alejandro Jueves' 'Ruta Alejandro Viernes' 'Ruta Alejandro Sabado'
 '🗞 Visitas periodicas Martin' '🗞 Visitas Periodicas Dario'
 '🗞 Visitas periodicas Alejandro' 'Clientes Especiales' '💲Creditos Hugo'
 '💲Creditos Martin' '💲Creditos Dario' '💲Creditos Alejandro'
 '💲Creditos Administracion']
